In [ ]:
import pandas as pd
import datetime
import yfinance as yf

df = pd.read_csv('StocksTraded.csv')
print(df.head())
print(df.shape)
df.columns = df.columns.str.strip()
symbols = df.Symbol
df.set_index('Symbol', inplace=True)

In [ ]:
print(df.loc['HDFCBANK']['LTP'])
columns = ['Stock', '30_days_ago', 'today']
filter_df = pd.DataFrame(columns=columns)

In [ ]:
columns = ['Stock', '10 days ago', 'today']
filter_df = pd.DataFrame(columns=columns)

for symbol in symbols:

    ticker = symbol+'.NS'
    end_date = datetime.datetime.now().date()
    start_date = end_date - datetime.timedelta(days=10)
    today_price = df.loc[symbol]['LTP']

    while True:
            
        # Fetch the historical data
        data = yf.download(ticker, start=start_date, end=end_date + datetime.timedelta(days=1))

        # Check if data is available for the start date
        if str(start_date) in data.index:
            value_10_days_ago = data.loc[str(start_date), 'Close']
            print(f"Stock value for {ticker}: {value_10_days_ago}")
            break
        else:
            start_date -= datetime.timedelta(days=1)

        new_entry = pd.DataFrame({'Stock':[ticker], '10 days ago':[value_10_days_ago], 'today':[today_price]})
        filter_df = pd.concat([filter_df, new_entry], ignore_index=True)


In [ ]:
days_ago_list = [0, 2, 5, 10, 30, 60, 90, 180, 365]

# Add new columns to df for each days_ago value
for days_ago in days_ago_list:
    df[f'{days_ago} Days Ago Price'] = 0.0

# Iterate over each stock symbol
for symbol in df.index:
    ticker = symbol + '.NS'
    end_date = datetime.datetime.now().date()
    current_price = df.loc[symbol]['LTP']

    # Fetch the historical data for the past year
    data = yf.download(ticker, start=end_date - datetime.timedelta(days=365), end=end_date + datetime.timedelta(days=1))

    # Loop through each days_ago value to find the price on that date
    for days_ago in days_ago_list:
        start_date = end_date - datetime.timedelta(days=days_ago)

        # Initialize days_ago_price
        days_ago_price = None

        # Loop to find the closest available date to the start_date
        while start_date >= (end_date - datetime.timedelta(days=365)):
            # Convert start_date to datetime for comparison
            if pd.Timestamp(start_date) in data.index:
                days_ago_price = data.loc[pd.Timestamp(start_date), 'Close']
                #print(f"Stock value for {ticker} on {start_date}: {days_ago_price}")
                break
            else:
                start_date -= datetime.timedelta(days=1)

        # If days_ago_price is not found, set a default value or handle accordingly
        if days_ago_price is None:
            #print(f"No data found for {ticker} within the last 365 days")
            days_ago_price = 0.0  # or some other default value or handling

        # Update the DataFrame with the days_ago price
        df.at[symbol, f'{days_ago} Days Ago Price'] = days_ago_price


In [ ]:

df.head()

# Identify rows with any 0 values
rows_with_zero = df.isin([0]).any(axis=1)

# Print the indices of rows with 0 values
print("Indices of rows with 0 values:", df[rows_with_zero].index.tolist())

# Drop rows with any 0 values
df = df[~rows_with_zero]

print(df.shape)
df['LTP'] = df['0 Days Ago Price']

In [ ]:
df['2_day_rise'] = -(df['2 Days Ago Price']-df['LTP'])*100/df['2 Days Ago Price']
df['5_day_rise'] = -(df['5 Days Ago Price'] - df['LTP'])*100/df['5 Days Ago Price']
df['1_month_rise'] = -(df['30 Days Ago Price'] - df['LTP'])*100/df['30 Days Ago Price']
df['2_month_rise'] = -(df['60 Days Ago Price'] - df['LTP'])*100/df['60 Days Ago Price']
df['3_month_rise'] = -(df['90 Days Ago Price']-df['LTP'])*100/df['90 Days Ago Price']
df['6_month_rise'] = -(df['180 Days Ago Price']-df['LTP'])*100/df['180 Days Ago Price']
df['1_year_rise']= -(df['365 Days Ago Price']-df['LTP'])*100/df['365 Days Ago Price']

In [ ]:
df.shape

In [ ]:
df.to_csv('stock_analysis.csv')

In [ ]:
import pandas as pd
df = pd.read_csv('stock_analysis.csv')

In [ ]:
print(df.columns)

In [ ]:
two_days_high_df = df.sort_values(by='2_day_rise', ascending=False)
five_days_high_df = df.sort_values(by='5_day_rise', ascending=False)
one_month_high_df = df.sort_values(by='1_month_rise', ascending=False)
two_month_high_df = df.sort_values(by='2_month_rise', ascending=False)
three_month_high_df = df.sort_values(by='3_month_rise', ascending=False)
six_month_high_df = df.sort_values(by='6_month_rise', ascending=False)
one_year_high_df = df.sort_values(by='1_year_rise', ascending=False)

In [ ]:
two_days_high_df = two_days_high_df[two_days_high_df['Series'] == 'EQ']
five_days_high_df = five_days_high_df[five_days_high_df['Series'] == 'EQ']
one_month_high_df = one_month_high_df[one_month_high_df['Series']=='EQ']
two_month_high_df = two_month_high_df[two_month_high_df['Series'] == 'EQ']
three_month_high_df = three_month_high_df[three_month_high_df['Series']=='EQ']
six_month_high_df = six_month_high_df[six_month_high_df['Series']=='EQ']
one_year_high_df = one_year_high_df[one_year_high_df['Series']=='EQ']


In [ ]:
one_month_two_day_equi = two_days_high_df[two_days_high_df['1_month_rise']>two_days_high_df['5_day_rise'] and df['5_day_rise']>0]

In [ ]:
one_month_two_day_equi[0:10]